In [2]:
# GAILを用いて、cameleon-gymの環境で学習する

import gymnasium as gym
import cameleon_gym_env
import time
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.data import rollout
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
env = gym.make('Cameleon-v0')

passing_point: 9.298374274579297,0.7140227782159305
passing_point: 18.439420674206403,0.7261115474247972
passing_point: 26.31022286030231,4.4299453195182235
passing_point: 18.241477220810264,3.4738049889871743
passing_point: 16.57837141989547,6.153300185164161
passing_point: 24.394152342891,11.44104159138658
passing_point: 30.627451513258972,17.948841119944245
passing_point: 35.546772097326354,24.55031853035083
passing_point: 32.33415367638262,33.45771076972119
passing_point: 23.710515027051564,28.493829968921727
passing_point: 32.16610495421365,31.255699101280264
passing_point: 22.84347246786236,31.270072566941735
passing_point: 31.16601322660081,33.57087165895269
passing_point: 29.05929971542152,24.948212470683785
passing_point: 35.61567152472278,18.82022739294436
passing_point: 44.201608820220684,15.569421677892072
passing_point: 46.45845968400796,25.09334780911007
passing_point: 39.896743806015884,31.494629707835745
passing_point: 36.17909573386219,23.40656400929269
passing_point: 

In [6]:
import pickle


SEED = 42

env = make_vec_env(
    "Cameleon-v0",
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
env_file_path = env.__module__
print(env_file_path)

path_file = "/media/gaia-22/ESD-ISO/auto_cosmos/cameleon/data/pkl/rollout02.pkl"
rollouts = pickle.load(open(path_file, 'rb'))

print(rollouts)
 
learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=5,
    seed=SEED,
)
reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)
gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=64,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=env,
    gen_algo=learner,
    reward_net=reward_net,
)

env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)
gail_trainer.train(200_000)

env.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)

print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)


passing_point: 8.711457760291292,2.938473955833401
passing_point: 12.777180902596847,4.182556346831265
passing_point: 13.803387883983694,4.4238959150705535
passing_point: 11.369887431994467,13.401709117705273
passing_point: 2.6701812252849244,9.726902791094705
passing_point: 9.119185539304237,14.46349083425169
passing_point: 12.048584175727953,23.22152517969327
passing_point: 7.893676550188336,30.074485424794762
passing_point: 14.037429325654756,36.71893007716522
passing_point: 6.760377081374394,33.176822993331854
passing_point: 5.607697871011738,41.56352338856897
passing_point: 0.5965609571850745,34.720634399438985
passing_point: 10.116393748212289,37.670743528931965
passing_point: 15.041568085251221,46.050405859711574
passing_point: 23.395920158771666,49.12165573537954
passing_point: 27.211421509998345,40.00335643044525
passing_point: 33.07528812111372,46.5258237527125
passing_point: 25.684013061097026,42.40624813841365
passing_point: 31.1335635091,50.02426813456201
passing_point: 25

KeyError: 'next_obs'